<a href="https://colab.research.google.com/github/aknip/Langchain-etc./blob/main/Langchain-Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchainhub langchain_experimental openai streamlit
!pip install google-search-results wikipedia

In [ ]:
from langchain.agents import Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain import hub
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.utilities import SerpAPIWrapper, SQLDatabase, WikipediaAPIWrapper
from langchain_experimental.sql import SQLDatabaseChain
from langchain.tools import WikipediaQueryRun
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.tools.render import format_tool_to_openai_function
import os
import langchain
import openai
import streamlit as st

In [ ]:
# Download "Chinook" Music Sales DB
import os
import requests
import zipfile
fname = 'chinook.zip'
url = 'https://www.sqlitetutorial.net/wp-content/uploads/2018/03/' + fname
r = requests.get(url)
open(fname, 'wb').write(r.content)
zipfile.ZipFile('chinook.zip').extractall()
assert os.path.exists('chinook.db')

In [ ]:
os.environ["OPENAI_API_KEY"] = ""
llm = ChatOpenAI(temperature=0, model='gpt-4-0613')
search = SerpAPIWrapper(serpapi_api_key="")
db = SQLDatabase.from_uri("sqlite:///chinook.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about real-time events. You should ask targeted questions"
    ),
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="useful for when you need to answer questions about a big picture or background of something."
    ),
    Tool(
        name="MusicSales",
        func=db_chain.run,
        description="useful for when you need to answer questions about mucis sales in a store. Should be strickly follow the tables info."
    )
]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a useful assistant."),
    MessagesPlaceholder(variable_name='chat_history'),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)


In [ ]:
# prompt = "How many people live in the US?"
# prompt = "What is the best selling song in the music store?"
# prompt = "What is the cheapest album in the music store? Tell me the price."
# prompt = "What is the best selling song starting with the letter 'F' in the music store? Tell me title and artist."
prompt = "How is the weather now in Cologne? Please use Celsius as unit for temperature."
response = agent_executor.invoke({"input": prompt})["output"]



> Entering new AgentExecutor chain...

Invoking: `Search` with `current weather in Cologne in Celsius`


{'type': 'weather_result', 'temperature': '13', 'unit': 'Celsius', 'precipitation': '2%', 'humidity': '77%', 'wind': '11 km/h', 'location': 'Cologne, Germany', 'date': 'Tuesday 10:00 PM', 'weather': 'Clear'}The current weather in Cologne, Germany is clear with a temperature of 13 degrees Celsius. The humidity is at 77% and there is a 2% chance of precipitation. The wind speed is 11 km/h.

> Finished chain.


In [ ]:
print(response)

The current weather in Cologne, Germany is clear with a temperature of 13 degrees Celsius. The humidity is at 77% and there is a 2% chance of precipitation. The wind speed is 11 km/h.


# Gradio Chatbot

In [ ]:
  !pip install gradio -q
  %load_ext gradio

In [25]:
def run_gradio_app():
  import gradio as gr
  import random
  import time

  # Theming
  theme = gr.themes.Default(
      primary_hue="slate" # , radius_size=gr.themes.Size(radius_sm="3px", radius_xs="2px", radius_xxs="1px")
  )
  # Styling: Change max width
  css = """
    .gradio-container {max-width: 800px!important}
    .vspacer1 {margin-top: 50px}
  """

  with gr.Blocks(theme=theme, css=css) as demo:

      gr.Markdown("# MyApp", elem_classes="vspacer1")
      gr.Markdown("### Optimizing your work with LLMs.")


      #
      # 1. Tab 1
      #
      with gr.Tab("Chat"):
        # https://www.gradio.app/docs/chatbot

        chatbot = gr.Chatbot(bubble_full_width=False)
        msg = gr.Textbox()
        clear = gr.ClearButton([msg, chatbot])


        def respond(message, chat_history):
            bot_message = random.choice(["How are you?", "I love you", "I'm very hungry"])
            chat_history.append((message, bot_message))
            #time.sleep(2)
            return "", chat_history

        def respond1(message, chat_history):
            chat_history.append((message, None))
            return "", chat_history

        def respond2(chat_history):
            bot_message = random.choice(["How are you?", "I love you", "I'm very hungry"])
            chat_history.append((None, bot_message))
            chat_history = [['hello', bot_message]] # hack
            time.sleep(2)
            return chat_history

        #msg.submit(respond, [msg, chatbot], [msg, chatbot])


        msg.submit(respond1, [msg, chatbot], [msg, chatbot], queue=False).then(
          respond2, chatbot, chatbot
        )


      #
      # 2. Tab 2
      #
      with gr.Tab("Input Text "):
        gr.Markdown("Please enter text")

        # Input text via UI
        gr.Markdown("### Input your text:")
        text_input = gr.Textbox(label="Enter text", placeholder="Your text here...", lines=10)
        text_output = gr.Textbox(label="Result")


      #
      # 3. Tab 3
      #
      with gr.Tab("Step 2"):
        gr.Markdown("Please select the optimization:")
        radio = gr.Radio(
          ["by headline", "by paragraph", "by §§§"], label="Text split method"
        )
        name = gr.Textbox(label="Name", placeholder="Enter text...")
        output = gr.Textbox(label="Output Box")


  demo.launch(quiet=True, share=False, debug=True)

run_gradio_app()

<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
